In [ ]:
#importing all the libraries
import numpy as np
import pandas as pd
from keras.datasets import fashion_mnist
from keras import optimizers
from keras.models import Sequential, load_model, clone_model
from keras.layers import Dense, Flatten

In [ ]:
#Loading the dataset
(X_train,y_train),(X_test,y_test) = fashion_mnist.load_data()

In [ ]:
X_train.shape

(60000, 28, 28)

In [ ]:
#Taking only 8 classes from fmnist dataset(except bag and ankle boots)
selected_classes = [0, 1, 2, 3, 4, 5, 6, 7]
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker']

In [ ]:
#Creating the dataset with 8 classes only
train_ind = np.isin(y_train, selected_classes)
test_ind = np.isin(y_test, selected_classes)

In [ ]:
print(train_ind)
print(test_ind)

[False  True  True ...  True  True  True]
[False  True  True ... False  True  True]


In [ ]:
X_train_A = X_train[train_ind]
y_train_A = y_train[train_ind]
X_test_A = X_test[test_ind]
y_test_A = y_test[test_ind]

In [ ]:
X_train_A[0][1]

array([  0,   0,   0,   1,   0,   0,   0,  49, 136, 219, 216, 228, 236,
       255, 255, 255, 255, 217, 215, 254, 231, 160,  45,   0,   0,   0,
         0,   0], dtype=uint8)

In [ ]:
print(X_train_A.shape)
print(y_train_A.shape)

(48000, 28, 28)
(48000,)


In [ ]:
#Converting the data type from int to float and also scalling the data between 0 and 1
X_train_A = X_train_A.astype('float32') / 255.0
X_test_A = X_test_A.astype('float32') / 255.0

In [ ]:
X_train_A[0][1]

array([0.        , 0.        , 0.        , 0.00392157, 0.        ,
       0.        , 0.        , 0.19215687, 0.53333336, 0.85882354,
       0.84705883, 0.89411765, 0.9254902 , 1.        , 1.        ,
       1.        , 1.        , 0.8509804 , 0.84313726, 0.99607843,
       0.90588236, 0.627451  , 0.1764706 , 0.        , 0.        ,
       0.        , 0.        , 0.        ], dtype=float32)

In [ ]:
#Creating model_A 
model_A = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(8, activation='softmax')
])

In [ ]:
#Training model A
model_A.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model_A.fit(X_train_A, y_train_A, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 8s 5ms/step - loss: 0.5162 - accuracy: 0.8107
Epoch 2/5
1500/1500 [==============================] - 13s 9ms/step - loss: 0.3963 - accuracy: 0.8513
Epoch 3/5
1500/1500 [==============================] - 8s 6ms/step - loss: 0.3620 - accuracy: 0.8627
Epoch 4/5
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3387 - accuracy: 0.8733
Epoch 5/5
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3210 - accuracy: 0.8796


In [ ]:
loss, accuracy = model_A.evaluate(X_test_A, y_test_A)

250/250 [==============================] - 1s 2ms/step - loss: 0.3814 - accuracy: 0.8602


In [ ]:
#Now creating dataset for two remaning classes i.e bag and ankle boot
name_of_classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker','Bag','Ankle boot']

In [ ]:
class_ind = [name_of_classes.index('Bag'), name_of_classes.index('Ankle boot')]

In [ ]:
X_train_B = X_train[(y_train == class_ind[0]) | (y_train == class_ind[1])]
y_train_B = y_train[(y_train == class_ind[0]) | (y_train == class_ind[1])]

In [ ]:
X_test_B = X_test[(y_test == class_ind[0]) | (y_test == class_ind[1])]
y_test_B = y_test[(y_test == class_ind[0]) | (y_test == class_ind[1])]

In [ ]:
X_train_B.shape

(12000, 28, 28)

In [ ]:
y_train_B

array([9, 9, 9, ..., 9, 9, 8], dtype=uint8)

In [ ]:
y_train_B = (y_train_B == class_ind[1]).astype(int)
y_test_B = (y_test_B == class_ind[1]).astype(int)

In [ ]:
y_test_B.shape

(2000,)

In [ ]:
#Transfer Learning
model_B_on_A = Sequential(model_A.layers[:-1])

In [ ]:
model_B_on_A.add(Dense(1, activation="sigmoid"))

In [ ]:
model_A_clone = clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

In [ ]:
for layer in model_B_on_A.layers[:-1]:
 layer.trainable = False

In [ ]:
model_B_on_A.compile(loss="binary_crossentropy", optimizer="sgd",
 metrics=["accuracy"])

In [ ]:
model_B_on_A.fit(X_train_B, y_train_B, epochs=5)

Epoch 1/5
375/375 [==============================] - 1s 2ms/step - loss: 21.6199 - accuracy: 0.9869
Epoch 2/5
375/375 [==============================] - 1s 2ms/step - loss: 8.6157 - accuracy: 0.9946
Epoch 3/5
375/375 [==============================] - 1s 2ms/step - loss: 9.5981 - accuracy: 0.9942
Epoch 4/5
375/375 [==============================] - 1s 2ms/step - loss: 7.6346 - accuracy: 0.9952
Epoch 5/5
375/375 [==============================] - 1s 3ms/step - loss: 6.3231 - accuracy: 0.9950


In [ ]:
for layer in model_B_on_A.layers[:-1]:
 layer.trainable = True

In [ ]:
optimizer = optimizers.SGD(lr=1e-4)
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer,metrics=["accuracy"])
model_B_on_A.fit(X_train_B, y_train_B, epochs=16)

Epoch 1/16
375/375 [==============================] - 2s 4ms/step - loss: 6.7856 - accuracy: 0.9958
Epoch 2/16
375/375 [==============================] - 1s 4ms/step - loss: 2.0427 - accuracy: 0.9984
Epoch 3/16
375/375 [==============================] - 1s 3ms/step - loss: 1.3355 - accuracy: 0.9982
Epoch 4/16
375/375 [==============================] - 1s 3ms/step - loss: 0.7696 - accuracy: 0.9988
Epoch 5/16
375/375 [==============================] - 1s 3ms/step - loss: 0.0748 - accuracy: 0.9998
Epoch 6/16
375/375 [==============================] - 1s 3ms/step - loss: 0.0779 - accuracy: 0.9997
Epoch 7/16
375/375 [==============================] - 1s 3ms/step - loss: 0.3543 - accuracy: 0.9994
Epoch 8/16
375/375 [==============================] - 1s 3ms/step - loss: 0.1093 - accuracy: 0.9998
Epoch 9/16
375/375 [==============================] - 1s 3ms/step - loss: 1.8267e-07 - accuracy: 1.0000
Epoch 10/16
375/375 [==============================] - 1s 3ms/step - loss: 2.1413e-08 - accuracy

In [ ]:
loss, accuracy = model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 2ms/step - loss: 10.7250 - accuracy: 0.9975
